In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

HUGGINGFACEHUB_API_TOKEN = "..."

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    provider="novita",
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
)

chat = ChatHuggingFace(llm=llm, verbose=True)


messages = [("system", "You are a helpful translator. Translate the user sentence to Spanish."), ("human", "I love programming."), ]
chat.invoke(messages)


# Test Agent Executor

In [6]:
from langchain_ollama import ChatOllama

from tools.network import ping_ip, query_dns_record, get_domain_metadata
from langchain_core.prompts import PromptTemplate
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

ModuleNotFoundError: No module named 'langchain_ollama'

In [ ]:
get_domain_metadata("bohiio.com")

C:\Users\enmanuel medina\AppData\Local\Temp\ipykernel_10412\2269374032.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  get_whois_info("bohiio.com")


WhoisRecord(domain_name='BOHIIO.COM', registrar='Squarespace Domains II LLC', whois_server='whois.squarespace.domains', updated_date=[datetime.datetime(2024, 7, 27, 16, 12, 50), datetime.datetime(2024, 7, 27, 16, 12, 50, 222609)], creation_date=datetime.datetime(2020, 8, 11, 2, 9, 55), expiration_date=datetime.datetime(2025, 8, 11, 2, 9, 55), name_servers=['CHAN.NS.CLOUDFLARE.COM', 'FELICITY.NS.CLOUDFLARE.COM'], status=['clientDeleteProhibited https://icann.org/epp#clientDeleteProhibited', 'clientTransferProhibited https://icann.org/epp#clientTransferProhibited', 'client transfer prohibited http://www.icann.org/epp#client transfer prohibited', 'client delete prohibited http://www.icann.org/epp#client delete prohibited'], emails=['abuse-complaints@squarespace.com'], dnssec='unsigned', name='REDACTED FOR PRIVACY', org='', country='DO')

In [ ]:
model="llama3.2:3b"
# network support tools
tools = [
    ping_ip,
    query_dns_record,
    get_domain_metadata,    
]

In [4]:
template = """
    You are a network support assistant. Your role is to help users diagnose and resolve networking issues by using the tools provided. You are knowledgeable in network protocols, diagnostics, configurations, and common issues related to connectivity, latency, DNS, firewalls, and more.
    
    Answer the following questions as best you can. You have access to the following tools:

    {tools}

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question. It must be on the same language as the input question.

    Begin!

    Question: {input}
    Thought: {agent_scratchpad}
"""
tool_descriptions = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])
tool_names = ", ".join([tool.name for tool in tools])
prompt_template = PromptTemplate(
    input_variables=["input"],
    template=template,
    partial_variables={"tools": tool_descriptions, "tool_names": tool_names},
)

In [5]:

llm = ChatOllama(model="llama3.2:3b", temperature=0)

agent = create_tool_calling_agent(llm, tools, prompt_template)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [6]:
response = agent_executor.invoke(input={"input": "Sabes de que pais es el dominio www.bohiio.com?"})



> Entering new AgentExecutor chain...

Invoking: `query_dns_record` with `{'domain_name': 'www.bohiio.com', 'record_type': 'SOA'}`


['No SOA records found.']
Invoking: `query_dns_record` with `{'domain_name': 'www.bohiio.com', 'record_type': 'SOA'}`


['No SOA records found.']
Invoking: `query_dns_record` with `{'domain_name': 'www.bohiio.com', 'record_type': 'SOA'}`


['No SOA records found.']
Invoking: `get_whois_info` with `{'domain': 'www.bohiio.com'}`


domain_name='BOHIIO.COM' registrar='Squarespace Domains II LLC' whois_server='whois.squarespace.domains' updated_date=[datetime.datetime(2024, 7, 27, 16, 12, 50), datetime.datetime(2024, 7, 27, 16, 12, 50, 222609)] creation_date=datetime.datetime(2020, 8, 11, 2, 9, 55) expiration_date=datetime.datetime(2025, 8, 11, 2, 9, 55) name_servers=['CHAN.NS.CLOUDFLARE.COM', 'FELICITY.NS.CLOUDFLARE.COM'] status=['clientDeleteProhibited https://icann.org/epp#clientDeleteProhibited', 'clientTransferProhibited https://icann.org/epp#clientTran

In [7]:
print("Response from the network support bot:")
print(type(response))
print(response["output"])

Response from the network support bot:
<class 'dict'>
Question: What is the country of origin for the domain BOHIIO.COM?

Answer: The country of origin for the domain BOHIIO.COM is DO (Dominican Republic). 

Note: The country code 'DO' is used to represent the Dominican Republic.


In [5]:
from tools.vectordb import init_vector_db

In [6]:
vectordb = init_vector_db()

Initializing Pinecone...
Using Pinecone index: network-support-chatbot
Initializing embeddings...
Creating Pinecone vector store...
Pinecone vector store initialized.


In [21]:
results = vectordb.max_marginal_relevance_search(query="What is OSPF?",k=3)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* CHAPTER 8
OSPF
This chapter covers the following subjects:
OSPF Fundamentals: This section provides an overview of communication between 
OSPF routers.
OSPF Configuration: This section describes the OSPF configuration techniques and 
commands that can be executed to verify the exchange of routes.
Default Route Advertisement: This section explains how default routes are advertised in 
OSPF.
Common OSPF Optimizations: This section reviews common OSPF settings for 
optimizing the operation of the protocol.
The Open Shortest Path First (OSPF) protocol is the first link-state routing protocol covered 
in this book. OSPF is a nonproprietary Interior Gateway Protocol (IGP) that overcomes the 
deficiencies of other distance vector routing protocols and distributes routing information 
within a single OSPF routing domain. OSPF introduced the concept of variable-length sub-
net masking (VLSM), which supports classless routing, summarization, authentication, and 
external route tagging. There a

In [22]:
a = vectordb.as_retriever()

In [24]:
a.invoke("What is OSPF?")[0].page_content

'CHAPTER 8\nOSPF\nThis chapter covers the following subjects:\nOSPF Fundamentals: This section provides an overview of communication between \nOSPF routers.\nOSPF Configuration: This section describes the OSPF configuration techniques and \ncommands that can be executed to verify the exchange of routes.\nDefault Route Advertisement: This section explains how default routes are advertised in \nOSPF.\nCommon OSPF Optimizations: This section reviews common OSPF settings for \noptimizing the operation of the protocol.\nThe Open Shortest Path First (OSPF) protocol is the first link-state routing protocol covered \nin this book. OSPF is a nonproprietary Interior Gateway Protocol (IGP) that overcomes the \ndeficiencies of other distance vector routing protocols and distributes routing information \nwithin a single OSPF routing domain. OSPF introduced the concept of variable-length sub-\nnet masking (VLSM), which supports classless routing, summarization, authentication, and \nexternal route t

In [1]:
from tools.vectordb import knowledge_base

c:\Users\enmanuel medina\Desktop\pii\VIU\trabajo fin master\network-support-chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from tools.vectordb import init_vector_db

vector_db = init_vector_db()


Initializing Pinecone...
Using Pinecone index: network-support-chatbot
Initializing embeddings...
Creating Pinecone vector store...
Pinecone vector store initialized.


In [16]:
vector_db.similarity_search_with_score("aon;wenwefnsdaxzjnw1nlnqascdsr1-03.", k=3)

[(Document(id='ed6b82c5-05c5-4080-842c-2f36134ea743', metadata={'author': '', 'creationDate': "D:20200109173217+03'30'", 'creationdate': '2020-01-09T17:32:17+03:30', 'creator': '', 'file_path': '/content/drive/My Drive/viu_train/CCNP-ENT-ENCOR.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': "D:20200109184033+03'30'", 'moddate': '2020-01-09T18:40:33+03:30', 'page': 0.0, 'producer': '', 'source': 'CCNP-ENT-ENCOR.pdf', 'subject': '', 'title': '', 'total_pages': 1073.0, 'trapped': ''}, page_content=''),
  0.856493533),
 (Document(id='7ed51669-6706-4ddf-895c-76f26ef10d1f', metadata={'author': '', 'creationDate': "D:20200109173217+03'30'", 'creationdate': '2020-01-09T17:32:17+03:30', 'creator': '', 'file_path': '/content/drive/My Drive/viu_train/CCNP-ENT-ENCOR.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': "D:20200109184033+03'30'", 'moddate': '2020-01-09T18:40:33+03:30', 'page': 940.0, 'producer': '', 'source': 'CCNP-ENT-ENCOR.pdf', 'subject': '', 'title': '', 'total_pages': 10

In [3]:
ans = knowledge_base("What is OSPF?", num_results=3)
for i, doc in enumerate(ans):
    print(f"{i+1}. {doc}")

1. CHAPTER 8
OSPF
This chapter covers the following subjects:
OSPF Fundamentals: This section provides an overview of communication between 
OSPF routers.
OSPF Configuration: This section describes the OSPF configuration techniques and 
commands that can be executed to verify the exchange of routes.
Default Route Advertisement: This section explains how default routes are advertised in 
OSPF.
Common OSPF Optimizations: This section reviews common OSPF settings for 
optimizing the operation of the protocol.
The Open Shortest Path First (OSPF) protocol is the first link-state routing protocol covered 
in this book. OSPF is a nonproprietary Interior Gateway Protocol (IGP) that overcomes the 
deficiencies of other distance vector routing protocols and distributes routing information 
within a single OSPF routing domain. OSPF introduced the concept of variable-length sub-
net masking (VLSM), which supports classless routing, summarization, authentication, and 
external route tagging. There 